In [1]:
import os
os.environ["mode"] = 'dev'

In [2]:

import sys
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from utils_ak.interactive_imports import *

In [3]:


def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def _load(module_obj, module_name=None, reload=False, import_globals=False):
    module = load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

def load_sources():
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/time.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/block.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/color.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/drawing.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/interval.py', import_globals=True, reload=True)
    
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\blocks.py', import_globals=True, reload=True)

    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\models.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\algo.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\style.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\validation.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\utils\openpyxl_wrapper.py', import_globals=True, reload=True)
#     _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\utils\excel_client.py', import_globals=True, reload=True)

load_sources()

In [4]:
remove('output.xlsx')
load_sources()
request = anyconfig.load(r"C:\Users\Mi\YandexDisk\Shared\2020 umalat\schedules\2020.12.04.json")
date = cast_datetime(request['Date'])
df = gen_request_df(request)
root = make_schedule(request, date)
wb = draw_workbook(root, mode='prod')
wb.save('_drawing.xlsx')
rename('_drawing.xlsx', 'output.xlsx')
open_file_in_os('output.xlsx')

Starting from 24 salt 48 24
Starting from 48 water 48 54
Starting from 54 salt 76 54
Starting from 76 water 76 82
Starting from 82 salt 90 82
Starting from 90 water 90 96
Starting from 96 salt 104 96
Starting from 104 water 104 110
Starting from 110 salt 118 110
Starting from 124 salt 118 124
Starting from 147 salt 118 147
Starting from 183 salt 118 183
Starting from 206 salt 118 206
Starting from 216 salt 118 216


KeyError: None

In [5]:
root

root (0, 339]
  boiling (48, 160]
    pouring (48, 74]
      block (48, 48]
        termizator (48, 54]
        pouring_name (54, 74]
      block (48, 48]
        pouring_and_fermenting (48, 56]
        soldification (56, 62]
        cutting (62, 69]
        pouring_off (69, 72]
        extra (72, 74]
    drenator (74, 118]
    melting_and_packing (118, 160]
      melting (118, 138]
        block (118, 118]
          serving (118, 124]
        block (118, 118]
          serving (118, 124]
          melting_label (124, 128]
          melting_name (128, 138]
        block (118, 118]
          serving (118, 124]
          melting_process (124, 138]
        block (118, 118]
          block (118, 124]
          cooling1 (124, 129]
          cooling2 (129, 139]
      packing_and_preconfiguration (139, 160]
        packing (139, 160]
          block (139, 139]
            packing_label (139, 142]
            packing_name (163, 181]
          block (160, 160]
            packing_brand (160, 18